# ニュースのカテゴリをBERTで予測するNotebook
データセットは<a href="https://www.rondhuit.com/download.html#ldcc">ライブドアニュースのデータセット</a>を使用します。  
ライブラリはTensorFlow(Keras)、Huggingfaceのtransformersです。  

プログラムを組むにあたっては、transformersの以下のドキュメントを参考にしています。  
https://huggingface.co/transformers/training.html  


In [ ]:
# ライブドアニュースコーパスのダウンロード & 展開
import requests

# ダウンロードして保存
url = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
r = requests.get(url, stream=True)

local_filename = "./ldcc-20140209/ldcc-20140209.tar.gz"
with open(local_filename, 'wb') as f:
    for chunk in r.iter_content(chunk_size=1024): 
        if chunk:
            f.write(chunk)
            
# 展開
with tarfile.open(local_filename) as tar:
    tar.extractall()

In [1]:
# 展開したファイルの読み込み
import requests
import os
import pandas as pd
import tensorflow as tf
import numpy as np

base_dir_name = "ldcc-20140209/text/"
all_dirs = [name for name in os.listdir(base_dir_name) if os.path.isdir(base_dir_name + name)]
all_files = [base_dir_name + dir_name + "/" + file_name for dir_name in all_dirs for file_name in os.listdir(base_dir_name + dir_name) if file_name != "LICENSE.txt"]

news = []
for file_path in all_files:
    category = file_path.split("/")[2]
    with open(file_path, "r") as f:
        text = f.read()
        (url, datetime_str, title, body) = text.split("\n", maxsplit=3)
        # タイトルと本文を合体させる
        news.append({"text":title + body, "category": category})

# dataframe にしておく
news_df = pd.DataFrame(news)
print(news_df.columns)

# カテゴリを数字に変更しておく
categories = pd.Categorical(news_df.category)
news_df['label'] = categories.codes
news_df = news_df.drop(columns=["category"])
display(news_df)

Index(['text', 'category'], dtype='object')


,text,label
0,友人代表のスピーチ、独女はどうこなしている？ もうすぐジューン・ブライドと呼ばれる６月。独女...,0
1,ネットで断ち切れない元カレとの縁 携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通...,0
2,相次ぐ芸能人の“すっぴん”披露 その時、独女の心境は？ 「男性はやっぱり、女性の“すっぴん”...,0
3,ムダな抵抗！？ 加齢の現実 ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは...,0
4,税金を払うのは私たちなんですけど！ 6月から支給される子ども手当だが、当初は子ども一人当たり...,0
...,...,...
7362,爆笑問題・田中裕二も驚く「ひるおび!」での恵俊彰の“天然”ぶり28日に放送された「JUNK...,8
7363,黒田勇樹のDV騒動 ネット掲示板では冷ややかな声も30日、元俳優の黒田勇樹が、妻の中村瑠衣に...,8
7364,サムスンのアンドロイド搭載カメラが韓国で話題に韓国のIT専門ニュースサイト「ブロター・ネット...,8
7365,米紙も注目したゲーム「竹島争奪戦」米紙「ウォール・ストリート・ジャーナル」は29日、「竹島争...,8


In [2]:
# dataframe から transformers の dataset に変換(あとでshuffle+train_test_splitを使えるからこっちを使う)
from datasets import Dataset
dataset = Dataset.from_pandas(news_df)
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 7367
})

In [3]:
# tokenizeする
from transformers import AutoTokenizer
bert_folder = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(bert_folder)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=200)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
    num_rows: 7367
})

In [4]:
# train と test に分けておく(このときついでにshuffleされる)
tokenized_datasets = tokenized_datasets.train_test_split(0.2)
tokenized_datasets

full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

# 使わないカラムを除去しておく
tf_train_dataset = full_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = full_eval_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_train_dataset

full_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
    num_rows: 5893
})

In [5]:
# 'attention_mask', 'input_ids', 'token_type_ids', labelを抽出してtensorflowのdataset形式にする
batch_size = 8
train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.batch(batch_size)

# eval側も同じく
eval_features = {x: tf_eval_dataset[x].to_tensor() for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(batch_size)

# save と load をするならこれを使う。
# tf.data.experimental.save(train_tf_dataset, "news_train_tf.ds")
# tf.data.experimental.save(eval_tf_dataset, "news_eval_tf.ds")

# train_tf_dataset = tf.data.experimental.load("news_train_tf.ds")
# eval_tf_dataset = tf.data.experimental.load("news_eval_tf.ds")

In [13]:
# BERT モデルをロードしてtraining実行
from transformers import TFAutoModelForSequenceClassification
print(bert_folder)
model = TFAutoModelForSequenceClassification.from_pretrained(bert_folder, num_labels=9)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=2)

cl-tohoku/bert-base-japanese-whole-word-masking


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
737/737 [==============================] - 429s 568ms/step - loss: 0.7790 - sparse_categorical_accuracy: 0.7591 - val_loss: 0.2464 - val_sparse_categorical_accuracy: 0.9206
Epoch 2/2
737/737 [==============================] - 419s 569ms/step - loss

### パラメータ変更によるスコアの影響メモ
max_length: 100 の場合
<pre>
737/737 [==============================] - 209s 274ms/step - loss: 0.8824 - sparse_categorical_accuracy: 0.7181 - val_loss: 0.2752 - val_sparse_categorical_accuracy: 0.9213
Epoch 2/3
737/737 [==============================] - 202s 274ms/step - loss: 0.1971 - sparse_categorical_accuracy: 0.9426 - val_loss: 0.2329 - val_sparse_categorical_accuracy: 0.9301
Epoch 3/3
737/737 [==============================] - 202s 274ms/step - loss: 0.0794 - sparse_categorical_accuracy: 0.9795 - val_loss: 0.2825 - val_sparse_categorical_accuracy: 0.9227
</pre>

max_length: 200 の場合
<pre>
737/737 [==============================] - 428s 567ms/step - loss: 0.8875 - sparse_categorical_accuracy: 0.7085 - val_loss: 0.2794 - val_sparse_categorical_accuracy: 0.9172
Epoch 2/3
737/737 [==============================] - 417s 566ms/step - loss: 0.1850 - sparse_categorical_accuracy: 0.9415 - val_loss: 0.2178 - val_sparse_categorical_accuracy: 0.9423
Epoch 3/3
737/737 [==============================] - 418s 567ms/step - loss: 0.0860 - sparse_categorical_accuracy: 0.9779 - val_loss: 0.3017 - val_sparse_categorical_accuracy: 0.9261
</pre>

In [7]:
# モデルを保存/ロード
from transformers import TFAutoModelForSequenceClassification
pretrained_model_path = "BERT_pretrained_for_livedoor_news_max_len_200"
# model.save_pretrained(pretrained_model_path)
model = TFAutoModelForSequenceClassification.from_pretrained(pretrained_model_path, num_labels=9)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at BERT_pretrained_for_livedoor_news_max_len_200.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [8]:
# predict してみる。eval データだけど。
pred = model.predict(eval_tf_dataset)
pred_labels = np.argmax(pred.logits, axis=1)
np.unique(pred_labels)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [9]:
# 数字のLabelからカテゴリ名に変換するdictionaryを作る
label2category = pd.DataFrame({"label": categories.codes, "category": categories}).drop_duplicates().set_index("label").to_dict()["category"]
label2category

{0: 'dokujo-tsushin',
 1: 'it-life-hack',
 2: 'kaden-channel',
 3: 'livedoor-homme',
 4: 'movie-enter',
 5: 'peachy',
 6: 'smax',
 7: 'sports-watch',
 8: 'topic-news'}

In [26]:
# カテゴリ名の予測値と正解をタイトル+本文とともに出力してみる
answer = tf_eval_dataset["label"].numpy()
pred_answer_df = pd.DataFrame({"pred": pred_labels, "answer": answer, "text": full_eval_dataset["text"]})
pred_answer_df["pred"] = pred_answer_df.pred.apply(lambda x: label2category[x])
pred_answer_df["answer"] = pred_answer_df.answer.apply(lambda x: label2category[x])
pred_answer_df[:20]

,pred,answer,text
0,topic-news,topic-news,NHK「ひるブラ」で放送事故が発生21日に放送されたNHKの紀行バラエティー番組「ひるブラ」...
1,movie-enter,movie-enter,ミスチルが書き下ろした主題歌とともに、初恋の出会いを綴る ひたむきな愛が運命をも変える！ 3...
2,smax,smax,【女子力アップスマホ！ドコモ×ファッションブランドコラボモデル「F-09D ANTEPRIM...
3,kaden-channel,kaden-channel,やはり気になる……ガイガーカウンターの購入を考える人は全体の17．2％【話題】福島第一原子力...
4,smax,smax,Microsoft、クラウドストレージサービス「SkyDrive」のAndroid向けアプリ...
5,kaden-channel,kaden-channel,グリーが世界へ！ 1億5千万人規模のプラットフォーム「GREE Platform」【話題】グ...
6,sports-watch,sports-watch,【Sports Watch】東原亜希“デスブログ”、スポーツ界を超越柔道家・井上康生の妻でタ...
7,it-life-hack,kaden-channel,ハンガーに掛けたままスピード乾燥！シャープ、プラズマクラスター洗濯機がパワーアップシャープは...
8,peachy,peachy,“神社コンシェルジュ”が選ぶ、2011年こそ結婚成就させるなら「芝大神宮」 2011年はスタ...
9,smax,smax,スタイラスとの相性抜群！コミケカタログを持ち歩ける「CC-Viewer」（利用編）【Wind...


In [28]:
# 予測が外れたニュースのみを表示してみる
pred_answer_df.loc[pred_answer_df.pred != pred_answer_df.answer]

,pred,answer,text
7,it-life-hack,kaden-channel,ハンガーに掛けたままスピード乾燥！シャープ、プラズマクラスター洗濯機がパワーアップシャープは...
50,topic-news,it-life-hack,行間を読む 翻訳のされ方で大きく印象が変わる言葉の難しさ先日来日した折に日本のテレビ局の音楽...
59,peachy,movie-enter,インタビュ—：ウィリー・ガーソン（モジー役）ウィリー・ガーソン：みなさん、アイルランドからい...
60,movie-enter,livedoor-homme,インタビュー：福山雅治「日本は奇跡の列島」歌手で俳優の福山雅治がナビゲーターを務めるNHKの...
79,movie-enter,peachy,ウォンビンインタビュー「お互い寄り添うように助け合う愛もある」直筆サイン特別プレゼント迫力の...
...,...,...,...
1349,dokujo-tsushin,livedoor-homme,ソーシャル通販サイトVIVA JAPAN にて junhashimoto 2012春夏コレク...
1358,dokujo-tsushin,livedoor-homme,［第1回］オトコのホンネ座談会“独女と非独女は何か違うの？”8月2日、独女通信がブログメディ...
1384,peachy,dokujo-tsushin,次のバケーションはトルコに決まり！イスタンブールとエーゲ海エリアを巡る、“Blue Turk...
1420,it-life-hack,smax,かわいいから大人買いしたい！邪魔なケーブルをすっきりできる「ドロイド君 ケーブルホルダー」【...
